In [ ]:
settings = Settings()
settings.freqs = np.arange(3, 25, 1)
settings.exp_list = ['catFR1']
settings.time_range = (-750, 750)
settings.bin_Hz = 250
settings.spos_ranges = [[i] for i in range(1,13)]
settings.Save('theta_no_behavioral.pkl')



def RunPhaseSpreadNoBeh(sub):
    import numpy as np

    from CMLTools import Settings, SubjectDataFrames, CMLReadDFRow, Locator, \
                      LogDFException, LogException, SpectralAnalysis

    try:
        error_suffix = 'nobeh_'+sub

        settings = Settings.Load('theta_no_behavioral.pkl')

        df_sub = SubjectDataFrames(sub)
        df_sub = df_sub[[s in settings.exp_list for s in df_sub['experiment']]]
        locmasks = []

        sess_cnt=0
        valid_sess = []
        for row in df_sub.itertuples():
            try:
                reader = CMLReadDFRow(row)
                locmask = Locator(reader).Hippocampus()
                locmasks.append(locmask)
                valid_sess.append(sess_cnt)
            except Exception as e:
                LogDFException(row, e, error_suffix)
                sess_cnt += 1
        mask_array = np.zeros(len(df_sub), dtype=bool)
        mask_array[valid_sess] = 1
        df_sub = df_sub[mask_array]

        sa = SpectralAnalysis(settings.freqs, dfs=df_sub, elec_masks=locmasks, \
            time_range=settings.time_range, bin_Hz=settings.bin_Hz,split_recall=False)
        results = sa.PhaseSpreadSerPos(settings.spos_ranges)

        np.save('spread_nobeh_sub_'+sub+'.npy', [settings.freqs,settings.spos_ranges, \
                                              results, sa.Ns])
        np.save('spos_recall_'+sub+'.npy', [sa.sposarr, sa.list_count])
    except Exception as e:
        LogException(e, error_suffix)
        return False

    return True

ClusterRun(RunPhaseSpreadNoBeh, subs_cathips)